**Install transformers**

In [1]:
!pip install transformers 

     |████████████████████████████████| 1.3MB 11.1MB/s 
     |████████████████████████████████| 1.1MB 51.4MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
     |████████████████████████████████| 2.9MB 47.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7046feafa6a121cd2a3c696fdd12566f08d0b77e71f24a163c3637ec49a0af97
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import numpy as np
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import datetime
import os
import gc
%load_ext tensorboard

**Load the dataset**

In [38]:
train_data, info = tfds.load('cnn_dailymail', split = 'train', data_dir = 'data/', with_info=True)
test_data = tfds.load('cnn_dailymail', split = 'test', data_dir = 'data/')

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset cnn_dailymail (data/cnn_dailymail/plain_text/3.0.0)
INFO:absl:Constructing tf.data.Dataset for split train, from data/cnn_dailymail/plain_text/3.0.0
INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset cnn_dailymail (data/cnn_dailymail/plain_text/3.0.0)
INFO:absl:Constructing tf.data.Dataset for split test, from data/cnn_dailymail/plain_text/3.0.0


**Fine tune the T5 model**

In [70]:
class T5Model(TFT5ForConditionalGeneration):
    def __init__(self, *args, log_dir=None, cache_dir= None, **kwargs):
        super().__init__(*args, **kwargs)
    
    @tf.function
    def train_step(self, data):
        x = data[0]
        y = x['labels']
        with tf.GradientTape() as tape:
            outputs = self(inputs = x['inputs'], attention_mask = x['attention_mask'], labels = y, training=True, return_dict=True)
            loss = outputs.loss
            logits = outputs.logits
            loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, self.trainable_variables)
            
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables)) 
        self.compiled_metrics.update_state(y, logits)
        metrics = {m.name: m.result() for m in self.metrics}
        metrics.update({'loss': loss})
        
        return metrics

    def test_step(self, data):
        x = data[0]
        y = x['labels']
        output = self(inputs = x['inputs'], attention_mask = x['attention_mask'], labels = y, training=False, return_dict=True)
        loss = output.loss
        logits = output.logits
        loss = tf.reduce_mean(loss)
        self.compiled_metrics.update_state(y, logits)
        metrics = {m.name: m.result() for m in self.metrics}
        metrics.update({'loss': loss})
        return metrics


**Parameters**

In [64]:
model_path = './t5/models'
log_path = './t5/logs'
config = {
    'batch_size' : 4,
    'epochs' : 1,
    'learning_rate' :1e-4,
    'max_len' : 512,
    'summary_len' : 150
}
data_size = {
    'train': 10000,
    'test' : 1000
}
params = {'source_len' : 512,
          'target_len' : 150,
          'batch_size' : 4
          }
tokenizer = T5Tokenizer.from_pretrained("t5-base")


**Encode and generate data**

In [56]:
class DataGenerator(tf.keras.utils.Sequence):
  def  __init__(self, data, tokenizer, mode, source_len, target_len, batch_size):
   self.data = data
   self.tokenizer = tokenizer
   self.source_len = source_len
   self.target_len = target_len
   self.batch_size = batch_size
   self.mode = mode

  def __len__(self):
     return int(np.ceil(data_size[self.mode]/self.batch_size))
    
  def __getitem__(self, index):
    dataset_batch = self.data.skip(self.batch_size).take(self.batch_size)
    encoded_batch = self.encode_data(dataset_batch, self.tokenizer, self.source_len , self.target_len)
    return encoded_batch

  def encode_data(self, data, tokenizer, source_len , target_len ):
    source = data.map(lambda text: 'summarize: ' + text['article'])
    source = list(map(lambda text: str(text,'utf-8'),list(tfds.as_numpy(source))))
    target = data.map(lambda text: text['highlights'])
    target = list(map(lambda text: str(text,'utf-8'),list(tfds.as_numpy(target))))

    batch_encoding = tokenizer.prepare_seq2seq_batch(
        src_texts = source,
        tgt_texts =  target,
        max_length= source_len, 
        max_target_length= target_len,
        padding = 'max_length',
        return_tensors = 'tf')
    batch_encoding['labels'] = tf.where(batch_encoding['labels']==tokenizer.pad_token_id, -100, batch_encoding['labels'])

    return {'inputs' : batch_encoding['input_ids'], 
            'attention_mask' : batch_encoding['attention_mask'],
            'labels':batch_encoding['labels']
            } 




In [65]:
training_data = DataGenerator(train_data.take(data_size['train']), tokenizer, 'train', **params)
validation_data = DataGenerator(test_data.take(data_size['test']), tokenizer, 'test', **params)

**Callbacks**

In [8]:
log_dir = log_path + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                                                     
checkpoint_filepath = model_path + "/" + "T5-{epoch:04d}-{val_loss:.4f}.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

callbacks = [tensorboard_callback, model_checkpoint_callback] 

**Create the T5 model**

In [72]:

def create_model():
  model = T5Model.from_pretrained("t5-base")
  optimizer = tf.keras.optimizers.Adam(lr=config['learning_rate'])
  metrics = tf.keras.metrics.SparseCategoricalAccuracy(name = 'accuracy')
  model.compile(optimizer=optimizer, metrics = metrics)
  return model
model = create_model()


All model checkpoint layers were used when initializing T5Model.

All the layers of T5Model were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5Model for predictions without further training.


**Train the model**

In [73]:

epochs = config['epochs']
model.fit( training_data, validation_data = validation_data, epochs = epochs)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2500/2500 [==============================] - 2139s 855ms/step - accuracy: 0.4892 - loss: 0.0098 - val_accuracy: 0.2750 - val_loss: 3.1186


**Save the model**

In [81]:
model.save_pretrained(model_path)

  adding: content/t5/ (stored 0%)
  adding: content/t5/.ipynb_checkpoints/ (stored 0%)
  adding: content/t5/models/ (stored 0%)
  adding: content/t5/models/tf_model.h5 (deflated 11%)
  adding: content/t5/models/config.json (deflated 64%)


In [85]:
!zip -r t5.zip /content/t5
from google.colab import files
files.download("/content/t5.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Download a test text**

In [89]:
from google.colab import files
uploaded = files.upload()

Saving testfile01.txt to testfile01.txt


In [93]:
with open('/content/test.txt', 'r') as f:
  source_txt = f.read()
source_txt = 'summarize: ' + source_txt 
#print (source_txt)

**Test the model**

In [94]:

source = tokenizer.prepare_seq2seq_batch(
      src_texts = source_txt,
      max_length=512, 
      padding = 'max_length',
      return_tensors = 'tf')
input_ids = source['input_ids']
attention_mask = source['attention_mask']
'''summary = model.generate(input_ids = input_ids,
                attention_mask = attention_mask,
                 max_length=150, num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True)'''
summary = model.generate(input_ids = input_ids,
                attention_mask = attention_mask, 
                max_length=150, 
                num_beams=3,
                no_repeat_ngram_size=2, 
                early_stopping=True)                          
decoded_summary = tokenizer.decode(summary.numpy()[0])
print ('Summary:\n', decoded_summary)



Summary:
 Riverhead High School on Long Island, New York. Jordan Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll were all suspended for one day because the 'Tebowing' craze was blocking the hallway and present a safety hazard to students.
